Extract data volume and revenue of each company

In [18]:
import pandas as pd
import re

# Load the CSV file
file_path = 'bianlian_web_scraped.csv'
df = pd.read_csv(file_path)

# Define a function to extract the relevant fields from the "Content" column
def extract_info(content):
    # Split content by lines
    lines = content.split('\n')
    
    # Extracting the company name from the first line
    company_name = lines[0].strip() if len(lines) > 0 else None
    
    # Extracting the company website from the second line
    website = lines[1].strip() if len(lines) > 1 else None

    # Check if the website starts with "https", otherwise set to None
    if website and not website.startswith("http"):
        website = None

    # Extracting the description from the third line
    description = lines[2].strip() if len(lines) > 2 else None
    
    # Extracting the revenue from the line that contains 'Revenue:'
    revenue = None
    data_volume = None
    data_type = None
    industry = None
    for line in lines:
        if 'Revenue:' in line:
            revenue = line.split('Revenue:')[-1].strip()
            
        if "Data volume" in line:
            data_volume = line.split('Data volume:')[-1].strip()
            
        if "Data Volume" in line:
            data_volume = line.split('Data Volume:')[-1].strip()

    return pd.Series([company_name, website, description, industry, revenue, data_volume, data_type])

# Apply the extraction function to the "Content" column
extracted_df = df['Content'].apply(extract_info)
extracted_df.columns = ['Company Name', 'Company Website', 'Description', 'Industry', 'Revenue', 'Data Volume', 'Data Type']

# Add the original content column to the extracted dataframe
extracted_df['Original Content'] = df['Content']

# Display the cleaned dataframe
display(extracted_df)

# Save the cleaned dataframe to a new CSV if needed
extracted_df.to_csv('bianlian_cleaning_part_one.csv', index=False)


,Company Name,Company Website,Description,Industry,Revenue,Data Volume,Data Type,Original Content
0,Benson Kearley IFG - Insurance Brokers & Finan...,http://bkifg.com,Benson Kearley IFG offers personal and commerc...,None,> $20 Millions,1.4 TB,None,Benson Kearley IFG - Insurance Brokers & Finan...
1,"Law Offices of Michael J Gurfinkel, Inc",https://gurfinkel.com,"The Law Offices of Michael J. Gurfinkel, Inc. ...",None,< $5 Millions,400 Gb,None,"Law Offices of Michael J Gurfinkel, Inc\nhttps..."
2,Stein Fibers,https://steinfibers.com,"Founded in 1976, Stein Fibers, Ltd. is a suppl...",None,None,None,None,Stein Fibers\nhttps://steinfibers.com\nFounded...
3,Majestic Metals,https://fathommfg.com/fathom-denver,Majestic Metals is a precision sheet metal fab...,None,$24.1 Millions,735 GB,None,Majestic Metals\nhttps://fathommfg.com/fathom-...
4,Ladov Law Firm,None,Revenue: <$5 Millions,None,<$5 Millions,105 GB,None,Ladov Law Firm\nLadov Law Firm PC is a company...
...,...,...,...,...,...,...,...,...
89,"Mayer Antonellis Jachowicz & Haranas, LLP",https://hkwg.com,"The Massachusetts law firm of Mayer & Haranas,...",None,> $5 Millions,170 Gb,None,"Mayer Antonellis Jachowicz & Haranas, LLP\nhtt..."
90,Shoma group,https://www.shomagroup.com/,Shoma Group is a real estate development compa...,None,> $20 Millions,2 Tb,None,Shoma group\nhttps://www.shomagroup.com/\nShom...
91,Lindsay Municipal Hospital,http://lindsayhospital.com,Lindsay Municipal Hospital proudly serves the ...,None,$13 Millions,None,None,Lindsay Municipal Hospital\nhttp://lindsayhosp...
92,"Palmer Construction Co., Inc",https://palmerconstruction.net,"Palmer Construction is a design-build, general...",None,> $13 Millions,475 Gb,None,"Palmer Construction Co., Inc\nhttps://palmerco..."


Extract data types of stolen data

In [13]:
import difflib
import pandas as pd

# Load the CSV file into a dataframe
file_path = 'bianlian_cleaning_part_one.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Define separate keyword lists for each column
keywords_dict = {
    "Financial Data": ["finance", "financial", "budget", "accounting"],
    "PII": ["personal", "identifiable", "pii"],
    "Customer and Client Data": ["customer", "client", "consumer"],
    "Employee and HR data": ["employee", "hr", "human resources", "hr data"],
    "Legal Documents": ["legal", "court", "casefile", "litigation"],
    "Intellectual Property and Trade Secrets": ["intellectual property", "trade secret", "ip", "design"],
    "Databases": ["database", "databases", "db"],
    "Sensitive Communication": ["communication", "email", "chat", "message", "correspondence"],
    "Healthcare Data": ["health", "medical", "healthcare"],
    "Utility and Infrastructure Data": ["utility", "infrastructure", "power", "water"]
}

# Initialize the new columns in the dataframe with False values
for col in keywords_dict:
    df[col] = False
    
df['Data Type'] = df['Data Type'].str.replace('\n', ' ', regex=False)


# Function to find the closest match for each category
def match_keywords(data_type, keywords_dict):
    matches = {col: False for col in keywords_dict}
    for col, keywords in keywords_dict.items():
        for keyword in keywords:
            if difflib.get_close_matches(data_type, keywords, cutoff=0.7):
                matches[col] = True
    return matches

# Apply the matching logic to the dataframe
def update_sensitive_columns_difflib(row):
    data_type = str(row['Data Type']).lower() if pd.notna(row['Data Type']) else ""
    
    # Split the 'Data Type' column by whitespace (in case entries are separated by newline or spaces)
    data_entries = data_type.split(' ')
    
    # Initialize column match flags
    column_matches = {col: False for col in keywords_dict}
    
    # Check each entry in the data type
    for entry in data_entries:
        entry_matches = match_keywords(entry.strip(), keywords_dict)
        
        # Update match flags for each column
        for col, match in entry_matches.items():
            if match:
                column_matches[col] = True
    
    # Update the row with the match results
    for col, match in column_matches.items():
        row[col] = match
    
    return row

# Apply the function to each row of the dataframe
df = df.apply(update_sensitive_columns_difflib, axis=1)

# Move the 'Original Content' column to the last position
original_content = df.pop('Original Content')
df['Original Content'] = original_content

# Display the updated dataframe to the user
df.head()
df.to_csv('bianlian_cleaning_part_two.csv', index=False)
